# Introduction to exact real and complex arithmetic with Calcium



*Author: Fredrik Johansson*

This notebook gives an introduction to Calcium (https://fredrikj.net/calcium/), using the builtin ctypes-based Python wrapper.

The source notebook file: https://github.com/fredrik-johansson/calcium/blob/master/doc/introduction.ipynb

To run this Notebook locally, you need to build Calcium and add `calcium/pycalcium` to your Python path.

*Caution*: the ctypes wrapper is incomplete, slow, and has some problems (e.g. memory leaks). It is mainly intended for testing the library. A more robust Cython interface should eventually be developed for "production use", along with a Julia interface. Calcium itself is also still experimental.

## Imports


In [1]:
# For convenience
from IPython.display import display

# Import the Calcium ctypes wrapper
# Don't "import *" at home -- this is for demonstration use only
from pyca import *

Quick example: Euler's identity $e^{\pi i} + 1 = 0$

In [2]:
exp(pi * i) + 1

0

## Basic types

The core types implemented in Calcium are the following:

* Symbolic expressions - `pyca.fexpr` (`fexpr_t` in C)
* Algebraic numbers - `pyca.qqbar` (`qqbar_t` in C)
* Calcium numbers / field elements - `pyca.ca` (`ca_t` in C)

The types have different roles, and it is important to understand their differences.

**Symbolic expressions** are the most flexible representation: they preserve the exact form of the input without performing any computation or simplification. For example, $\sqrt{2} / 2$, $1 / \sqrt{2}$ and $2^{1/2} / 2$ are all different as expressions (although they represent the same number). Symbolic expressions are not very useful on their own for computations, but they are convenient as input and output for other, more "computational" types.

**Algebraic numbers** represent elements of the field $\overline{\mathbb{Q}}$ in *canonical form*, consisting of the number's minimal polynomial together with an isolating complex interval for a unique root. Thus one number can only have one representation: $\sqrt{2} / 2$ and $1 / \sqrt{2}$ will evaluate to exactly the same `qqbar` algebraic number. This is useful because the results are predictable and in a sense maximally simplified (contrary to the other types, it is impossible to construct a complicated `qqbar` instance that represents 0 without trivially being 0). The downsides are that field operations are much more expensive than in the `ca` representation, and in passing from a symbolic expression to a `qqbar`, structural information (such as a simple closed form) may be lost.

**Calcium numbers** represent numbers as elements of fields $\mathbb{Q}(a_1,\ldots,a_n)$. Calcium constructs extension numbers $a_k$ and fields $\mathbb{Q}(a_1,\ldots,a_n)$ automatically and lazily, so from the point of view of the user Calcium numbers behave just like numbers. The extension numbers $a_k$ can be "absolute" algebraic numbers (`qqbar` instances), but they can also be transcendental numbers like $\pi$ or $\exp(\sqrt{2} i)$. This representation is highly efficient for arithmetic, but in general does not guarantee a canonical form. Relations between extension elements (e.g. $\log(4) / \log(2) = 2$) are simplified automatically using ideal reduction, but Calcium will not be able to prove all relations, so it is possible to have a `ca` instance that represents 0 without trivially being 0 (such instances are nevertheless handled in a mathematically rigorous way as discussed below). The `ca` type can also represent special values and metavalues (signed and unsigned infinities, undefined, unknown).

## Symbolic expressions

In the `pyca` namespace, lowercase names (example: `sqrt`) denote `ca` functions while while uppercase names (example: `Sqrt`) denote `fexpr` symbolic expressions. It is easy to construct symbolic expressions:

In [3]:
# Create a symbolic sum of three terms
expr = Add(Sqrt(2) / 2, 1 / Sqrt(2), 2 ** Div(1, 2) / 2)
# Display the formula
expr

Add(Div(Sqrt(2), 2), Div(1, Sqrt(2)), Div(Pow(2, Div(1, 2)), 2))

In [4]:
type(expr)

pyca.fexpr

### Evaluating expressions and displaying objects

Constant expressions can be passed as input to other types, resulting in evaluation:

In [5]:
ca(expr)     # evaluate expr using ca arithmetic, producing a ca

2.12132 {(3*a)/2 where a = 1.41421 [a^2-2=0]}

In [6]:
type(_)

pyca.ca

In [7]:
qqbar(expr)     # evaluate expr using qqbar arithmetic, producing a qqbar

2.12132 (deg 2)

In [8]:
type(_)

pyca.qqbar

Symbolic expressions are also generated automatically behind the scenes in order to display objects in LaTeX in the notebook (this is done in the examples above). You can of course also print objects in text form:

In [9]:
print(expr)
print(ca(expr))
print(qqbar(expr))

Add(Div(Sqrt(2), 2), Div(1, Sqrt(2)), Div(Pow(2, Div(1, 2)), 2))
2.12132 {(3*a)/2 where a = 1.41421 [a^2-2=0]}
2.12132 (deg 2)


By default, `qqbar` objects display as polynomial roots. To produce a closed-form expression (if possible), the `qqbar.fexpr()` method can be called:

In [10]:
qqbar(expr).fexpr()

Div(Mul(3, Sqrt(2)), 2)

### Manipulating symbolic expressions

The `fexpr` type provides methods for rudimentary manipulation (accessing subexpressions and so on).

In [11]:
expr = Add(Sqrt(2) / 2, 1 / Sqrt(2), 2 ** Div(1, 2) / 2)
print(expr.head())
print(expr.nargs())
print(expr.args())
print(expr.is_atom())
print(expr.args()[0].args()[1].is_atom())
print(expr.num_leaves())
print(expr.nwords())
print(expr.replace(2, Pi))

Add
3
(Div(Sqrt(2), 2), Div(1, Sqrt(2)), Div(Pow(2, Div(1, 2)), 2))
False
True
16
24
Add(Div(Sqrt(Pi), Pi), Div(1, Sqrt(Pi)), Div(Pow(Pi, Div(1, Pi)), Pi))


The `.nstr()` method computes a numerical approximation using Arb, returning a decimal string:

In [12]:
print((Sqrt(2) / 2).nstr(30))
print((Exp(1 + 2j).nstr(10)))

# No symbolic simplification is done - Arb can generally not detect
# exact zeros, and zeros will be output in the form 0e-n
print((Exp(Pi*NumberI)).nstr(30))
print((Sqrt(2)/2 - 1/Sqrt(2)).nstr())

0.707106781186547524400844362105
-1.131204384 + 2.471726672*I
-1.00000000000000000000000000000 + 0e-37*I
0e-731


The `.expanded_normal_form()` method puts the given formula in a canonical form as a formal rational expression.

In [13]:
x = fexpr("x"); y = fexpr("y")
A = (x+y)**5 * (x-y) * (x + 1)
B = (x**2 - y**2) * (x**2 - 1)
(A / B).expanded_normal_form()

Div(Add(Pow(x, 4), Mul(4, Pow(x, 3), y), Mul(6, Pow(x, 2), Pow(y, 2)), Mul(4, x, Pow(y, 3)), Pow(y, 4)), Add(x, -1))

Please note that `.expanded_normal_form()` only simplifies rational arithmetic operations, treating anything non-arithmetical as an atomic node. For example, square roots are treated as atomic. It also does not simplify nodes recursively.

In [14]:
display((Sqrt(x) / Sqrt(x)).expanded_normal_form())
display((Sqrt(x) * Sqrt(x)).expanded_normal_form())
display((Sqrt(2*x) / Sqrt(x*2)).expanded_normal_form())

1

Pow(Sqrt(x), 2)

Div(Sqrt(Mul(2, x)), Sqrt(Mul(x, 2)))

We will not do anything more sophisticated with symbolic expressions in this notebook; we will instead move on to describing exact numerical calculations using the `ca` and `qqbar` types.

## Calcium numbers

Calcium numbers encompass rational numbers, of course:

In [15]:
ca(1) / 3

0.333333 {1/3}

In [16]:
(ca(1) / 3) * 6

2

Irrational numbers result in extensions of $\mathbb{Q}$:

In [17]:
# Alternative syntax: ca(2).sqrt()
sqrt(2)

1.41421 {a where a = 1.41421 [a^2-2=0]}

In [18]:
sqrt(2) ** 2

2

In [19]:
2 * pi

6.28319 {2*a where a = 3.14159 [Pi]}

Field arithmetic produces numbers represented as formal fraction field elements:

In [20]:
pi * i * sqrt(2)

4.44288*I {a*b*c where a = 3.14159 [Pi], b = 1.41421 [b^2-2=0], c = I [c^2+1=0]}

In [21]:
(pi * i * sqrt(2)) ** 2   # note the simplifications

-19.7392 {-2*a^2 where a = 3.14159 [Pi], b = 1.41421 [b^2-2=0], c = I [c^2+1=0]}

In [22]:
((pi + i + sqrt(2)) / (pi + sqrt(2)))**3

0.855459 + 0.647925*I {(a^3+3*a^2*b+3*a^2*c+6*a*b*c+3*a-b+5*c)/(a^3+3*a^2*b+6*a+2*b) where a = 3.14159 [Pi], b = 1.41421 [b^2-2=0], c = I [c^2+1=0]}

### Some more number field arithmetic

Let us construct the golden ratio:

In [23]:
phi = (sqrt(5) + 1) / 2
phi

1.61803 {(a+1)/2 where a = 2.23607 [a^2-5=0]}

We compute the 200th Fibonacci number using Binet's formula:

In [24]:
(phi**200 - (1-phi)**200) / sqrt(5)

2.80571e+41 {280571172992510140037611932413038677189525}

Depending on the operations, `ca` arithmetic may result in different field representations of the same number:

In [25]:
display(sqrt(2)*sqrt(3))
display(sqrt(6))

2.44949 {a*b where a = 1.73205 [a^2-3=0], b = 1.41421 [b^2-2=0]}

2.44949 {a where a = 2.44949 [a^2-6=0]}

The difference simplifies to zero:

In [26]:
display(sqrt(2)*sqrt(3) - sqrt(6))
display(sqrt(2)*sqrt(3) == sqrt(6))

0

True

Calcium will attempt to eliminate more complex extension numbers when it encounters extension numbers that are algebraically dependent. Here, it eliminates $\sqrt{6}$ from the expression, writing the result in terms of $\sqrt{2}$ and $\sqrt{3}$

In [27]:
sqrt(2)*sqrt(3) + sqrt(6)

4.89898 {2*b*c where a = 2.44949 [a^2-6=0], b = 1.73205 [b^2-3=0], c = 1.41421 [c^2-2=0]}

(Implementation detail: the output shows that $\sqrt{6}$ is still kept as part of the field structure, to aid future simplifications, although it is unused in this particular field element.)

In many cases, it would be desirable to write the above result as an element of the simple algebraic number field $\mathbb{Q}(\sqrt{6})$ or $\mathbb{Q}(\sqrt{2} + \sqrt{3})$ instead. Calcium will always stay within a univariate field when performing field operations starting from a single extension number, but it will not automatically reduce multivariate number fields to univariate fields. In the future, Calcium will offer more customizability so that the user can choose between different behaviors concerning simplification of extension numbers and fields.

### Predicates; contexts objects

Predicates in Calcium have mathematically semantics: they return True or False only if Calcium can prove the result. When the truth value is unknown, Calcium says so explicitly; in `pyca`, this is done by raising an exception (`NotImplementedError`). Consider, as an example, testing whether $\exp(\varepsilon) = 1$ where $\varepsilon$ is a small number. With $\varepsilon = 10^{-1000}$, Calcium finds that the numbers are not equal:

In [28]:
eps = ca(10) ** (-1000)
exp(eps) == 1

False

With $\varepsilon = 10^{-10000}$, Calcium fails:

In [29]:
eps = ca(10) ** (-10000)
try:
    exp(eps) == 1
except Exception as e:
    print(e)

unable to decide predicate: equal


The comparison fails because the internal precision limit for numerical evaluation has been exceeded. The precision limit and many other settings are stored in a context object, which also serves as a cache of computed data (such as extension numbers and extension fields). The context object has type `ca_ctx`. There is a default context called `ctx_default`, with the following settings:

In [30]:
ctx_default

ca_ctx(verbose=0, print_flags=3, mpoly_ord=0, prec_limit=4096, qqbar_deg_limit=120, low_prec=64, smooth_limit=32, lll_prec=128, pow_limit=20, use_gb=1, gb_length_limit=100, gb_poly_length_limit=1000, gb_poly_bits_limit=10000, vieta_limit=6)

If we create a new context object with higher `prec_limit` than the default value of 4096 bits, the computation succeeds:

In [31]:
ctx = ca_ctx(prec_limit=65536)
eps = ca(10, context=ctx) ** (-10000)
exp(eps) == 1

False

The intention is that the user will be able to create multiple "Calcium fields" for different purposes. Right now, context objects support only limited configurability.

## Algebraic number identities

Calcium implements a complete decision procedure for testing equality (or inequality) of algebraic numbers.  This functionality is accessible using either `qqbar` or `ca` operations.

Rob Corless proposed checking the following identity from Bill Gosper. We will first construct the LHS and RHS as symbolic expressions. We could evaluate them directly with `qqbar` or `ca` operations, which would be more efficient, but this way we can print the input.

In [32]:
I = NumberI

lhs = Sqrt(36 + 3*(-54+35*I*Sqrt(3))**Div(1,3)*3**Div(1,3) + \
            117/(-162+105*I*Sqrt(3))**Div(1,3))/3 + \
            Sqrt(5)*(1296*I+840*Sqrt(3)-35*3**Div(5,6)*(-54+35*I*Sqrt(3))**Div(1,3)-\
            54*I*(-162+105*I*Sqrt(3))**Div(1,3)+13*I*(-162+105*I*Sqrt(3))**Div(2,3))/(5*(162*I+105*Sqrt(3)))

rhs = Sqrt(5) + Sqrt(7)

display(Equal(lhs, rhs))

Equal(Add(Div(Sqrt(Add(Add(36, Mul(Mul(3, Pow(Add(-54, Mul(Mul(35, NumberI), Sqrt(3))), Div(1, 3))), Pow(3, Div(1, 3)))), Div(117, Pow(Add(-162, Mul(Mul(105, NumberI), Sqrt(3))), Div(1, 3))))), 3), Div(Mul(Sqrt(5), Add(Sub(Sub(Add(Mul(1296, NumberI), Mul(840, Sqrt(3))), Mul(Mul(35, Pow(3, Div(5, 6))), Pow(Add(-54, Mul(Mul(35, NumberI), Sqrt(3))), Div(1, 3)))), Mul(Mul(54, NumberI), Pow(Add(-162, Mul(Mul(105, NumberI), Sqrt(3))), Div(1, 3)))), Mul(Mul(13, NumberI), Pow(Add(-162, Mul(Mul(105, NumberI), Sqrt(3))), Div(2, 3))))), Mul(5, Add(Mul(162, NumberI), Mul(105, Sqrt(3)))))), Add(Sqrt(5), Sqrt(7)))

We can check numerically that the expressions agree:

In [33]:
print(lhs.nstr()); print(rhs.nstr())

4.881819288564380 - 0e-20*I
4.881819288564380


Evaluating the expressions as `qqbar`s gives the same result, proving the identity: 

In [34]:
display(qqbar(lhs)); display(qqbar(rhs))

4.88182 (deg 4)

4.88182 (deg 4)

Explicitly:

In [35]:
qqbar(lhs) == qqbar(rhs)

True

We can also perform the verification using `ca` arithmetic. The equality is not immediately apparent since the two expressions evaluate to elements of different extension fields of $\mathbb{Q}$:

In [36]:
display(ca(lhs))

4.88182 + 0e-33*I {(5*a-d^2*f^2*g+24*d*f*g-39*g)/(15*d*f) where a = 35.7176 + 34.3693*I [Sqrt(94.5000 + 2455.18*I {9*d^3+36*d^2*f^2+117*d*f})], b = 1.50000 + 38.9711*I [Pow(-162.000 + 181.865*I {105*h*i-162}, 0.666667 {2/3})], c = 4.50000 + 4.33013*I [Pow(-162.000 + 181.865*I {105*h*i-162}, 0.333333 {1/3})], d = 3.12013 + 3.00234*I [Pow(-54.0000 + 60.6218*I {35*h*i-54}, 0.333333 {1/3})], e = 2.49805 [Pow(3, 0.833333 {5/6})], f = 1.44225 [Pow(3, 0.333333 {1/3})], g = 2.23607 [g^2-5=0], h = 1.73205 [h^2-3=0], i = I [i^2+1=0]}

In [37]:
display(ca(rhs))

4.88182 {a+b where a = 2.64575 [a^2-7=0], b = 2.23607 [b^2-5=0]}

Fortunately, the equality operator manages to prove that the values are really equal:

In [38]:
ca(lhs) == ca(rhs)

True

Indeed, in this case `ca` arithmetic simplifies the difference of the expressions to 0 automatically:

In [39]:
ca(lhs) - ca(rhs)

0

If this example did not look challenging enough, you may try proving equality of the two huge expressions (involving 7000 operations) given in https://ask.sagemath.org/question/52653/equality-of-algebraic-numbers-given-by-huge-symbolic-expressions/. You will need a bit of string preprocessing to convert the given Sage expressions to `fexpr` expressions or ordinary Python syntax for use with the `ca` or `qqbar` types.

This test problem is implemented in the `huge_expr.c` program in the Calcium examples directory. It should take a few seconds to run.

## Transcendental number identities

Calcium is capable of manipulating numbers involving some transcendental functions such as `exp` and `log`. Contrary to the case of algebraic numbers, it does not have a complete decision procedure for transcendental numbers, but it has some decent heuristics.

### Exp and log numbers

Basic logarithmic and exponential simplifications work as expected:

In [40]:
log(10**20) / log(100)

10

In [41]:
exp(pi) * exp(-pi + log(2))

2

Different formulas for the same number may result in different internal field representations, in which case Calcium may yet be able to recognize relations when the numbers are subtracted or divided or when evaluating a predicate:

In [42]:
display(log(sqrt(2)))
display(log(2)/2)
display(log(sqrt(2)) - log(2)/2)
display(log(sqrt(2)) / (log(2)/2))
display(log(sqrt(2)) * (log(2)/2))
log(sqrt(2)) == log(2)/2

0.346574 {a where a = 0.346574 [Log(1.41421 {b})], b = 1.41421 [b^2-2=0]}

0.346574 {(a)/2 where a = 0.693147 [Log(2)]}

0

1

0.120113 {(b^2)/4 where a = 0.346574 [Log(1.41421 {c})], b = 0.693147 [Log(2)], c = 1.41421 [c^2-2=0]}

True

Calcium is aware of branch cuts:

In [43]:
# exp(log(x)) == x
display(exp(log(1 + 10*i)))
# log(exp(x)) != x in general; here we get the correct branch!
display(log(exp(1 + 10*i)))

1.00000 + 10.0000*I {10*a+1 where a = I [a^2+1=0]}

1.00000 - 2.56637*I {-4*a*b+10*b+1 where a = 3.14159 [Pi], b = I [b^2+1=0]}

Let us check the formulas given on the Calcium documentation front page:

In [44]:
log(sqrt(2)+sqrt(3)) / log(5 + 2*sqrt(6))

0.500000 {1/2}

In [45]:
i**i - exp(pi / ((sqrt(-2)**sqrt(2)) ** sqrt(2)))

0

In [46]:
ca(10)**-30 < (640320**3 + 744)/exp(pi*sqrt(163)) - 1 < ca(10)**-29

True

### Trigonometric functions

Calcium does not yet have trigonometric or inverse trigonometric funtions builtin at the C level, but we can "fake" these functions using complex exponentials and logarithms. The functions `pyca.sin`, `pyca.cos`, `pyca.tan` and `pyca.atan` do precisely this.

In [47]:
sin(3)

0.141120 - 0e-34*I {(-a^2*b+b)/(2*a) where a = -0.989992 + 0.141120*I [Exp(3.00000*I {3*b})], b = I [b^2+1=0]}

Calcium is capable of proving some trigonometric identities:

In [48]:
sin(sqrt(2)/2)**2 + cos(1/sqrt(2))**2

1

In [49]:
sin(3 + pi) + sin(3)

0

In [50]:
tan(atan(5)) == 5

True

In [51]:
try:
    atan(tan(1)) == 1
except NotImplementedError:
    print("This simplification does not work yet")

This simplification does not work yet


We test some simplifications involving the Gudermannian function $\operatorname{gd}(x) = 2 \operatorname{atan}(e^x) - \pi/2$:

In [52]:
def gd(x):
    return 2*atan(exp(x))-pi/2

display(sin(gd(1)) - tanh(1))
display(tan(gd(1)) - sinh(1))
display(sin(gd(sqrt(2))) - tanh(sqrt(2)))
display(tan(gd(1)/2) - tanh(ca(1)/2))

0

0

0

0

Let us try to prove a famous identity; Machin's formula for $\pi$:

In [53]:
lhs = 4*Atan(Div(1,5)) - Atan(Div(1,239))
rhs = Pi / 4
Equal(lhs, rhs)

Equal(Sub(Mul(4, Atan(Div(1, 5))), Atan(Div(1, 239))), Div(Pi, 4))

In [54]:
print(lhs.nstr())
print(rhs.nstr())

0.7853981633974483
0.7853981633974483


Evaluating the left-hand side using `ca` arithmetic gives us nothing as simple as $\pi / 4$:

In [55]:
4*atan(ca(1)/5) - atan(ca(1)/239)

0.785398 + 0e-34*I {(a*c-4*b*c)/2 where a = 0e-35 + 0.00836815*I [Log(0.999965 + 0.00836805*I {(239*c+28560)/28561})], b = 0e-34 + 0.394791*I [Log(0.923077 + 0.384615*I {(5*c+12)/13})], c = I [c^2+1=0]}

Nevertheless, Calcium finds the identity when $\pi$ is given as part of the input:

In [56]:
4*atan(ca(1)/5) - atan(ca(1)/239) - pi/4

0

In [57]:
4*atan(ca(1)/5) - atan(ca(1)/239) == pi/4

True

Here is a more complicated formula:

In [58]:
12*atan(ca(1)/49) + 32*atan(ca(1)/57) - 5*atan(ca(1)/239) + 12*atan(ca(1)/110443)

0.785398 + 0e-33*I {(-12*a*e+5*b*e-32*c*e-12*d*e)/2 where a = 0e-35 + 1.81089e-5*I [Log(1.00000 + 1.81089e-5*I {(110443*e+6098828124)/6098828125})], b = 0e-35 + 0.00836815*I [Log(0.999965 + 0.00836805*I {(239*e+28560)/28561})], c = 0e-34 + 0.0350841*I [Log(0.999385 + 0.0350769*I {(57*e+1624)/1625})], d = 0e-35 + 0.0408107*I [Log(0.999167 + 0.0407993*I {(49*e+1200)/1201})], e = I [e^2+1=0]}

In [59]:
12*atan(ca(1)/49) + 32*atan(ca(1)/57) - 5*atan(ca(1)/239) + 12*atan(ca(1)/110443) - pi/4

0

Hyperbolic formulas also work:

In [60]:
atanh = lambda x: -i*atan(i*x)
32*atanh(ca(1)/31) + 24*atanh(ca(1)/49) + 14*atanh(ca(1)/161)

1.60944 {-7*a-12*b-16*c where a = -0.0124225 [Log(0.987654 {80/81})], b = -0.0408220 [Log(0.960000 {24/25})], c = -0.0645385 [Log(0.937500 {15/16})], d = I [d^2+1=0]}

In [61]:
32*atanh(ca(1)/31) + 24*atanh(ca(1)/49) + 14*atanh(ca(1)/161) - log(5)

0

## Matrices

The `ca_mat` type provides matrices with `ca` entries. We look at some examples of basic manipulation:

In [62]:
A = ca_mat([[1, i*pi], [-i*pi, 2]])
A

ca_mat of size 2 x 2
[                                                        1, 3.14159*I {a*b where a = 3.14159 [Pi], b = I [b^2+1=0]}]
[-3.14159*I {-a*b where a = 3.14159 [Pi], b = I [b^2+1=0]},                                                       2]

In [63]:
A * A

ca_mat of size 2 x 2
[    10.8696 {a^2+1 where a = 3.14159 [Pi], b = I [b^2+1=0]}, 9.42478*I {3*a*b where a = 3.14159 [Pi], b = I [b^2+1=0]}]
[-9.42478*I {-3*a*b where a = 3.14159 [Pi], b = I [b^2+1=0]},   13.8696 {a^2+4 where a = 3.14159 [Pi], b = I [b^2+1=0]}]

In [64]:
display(A.det())
display(A.trace())
display(A.rank())

-7.86960 {-a^2+2 where a = 3.14159 [Pi], b = I [b^2+1=0]}

3

2

Solving linear systems:

In [65]:
B = ca_mat([[1], [2]])
X = A.solve(B)
display(X)
display(A * X)

ca_mat of size 2 x 1
[-0.254142 + 0.798412*I {(2*a*b-2)/(a^2-2) where a = 3.14159 [Pi], b = I [b^2+1=0]}]
[ -0.254142 - 0.399206*I {(-a*b-2)/(a^2-2) where a = 3.14159 [Pi], b = I [b^2+1=0]}]

ca_mat of size 2 x 1
[1]
[2]

Computing row echelon forms and characteristic polynomials:

In [66]:
ca_mat([[1,pi,2,pi],[1/pi,3,1/(pi+1),4],[1,1,1,1]]).rref()

ca_mat of size 3 x 4
[1, 0, 0, 0.401081 {(a^3-a^2-2*a)/(3*a^2+3*a-2) where a = 3.14159 [Pi]}]
[0, 1, 0,  1.35134 {(4*a^2+4*a-2)/(3*a^2+3*a-2) where a = 3.14159 [Pi]}]
[0, 0, 1,     -0.752416 {(-a^3+a)/(3*a^2+3*a-2) where a = 3.14159 [Pi]}]

In [67]:
A = ca_mat([[5, pi], [1, -1]])**4
display(A)
display(A.charpoly())
display(A.charpoly()(A))

ca_mat of size 2 x 2
[842.215 {a^2+66*a+625 where a = 3.14159 [Pi]}, 405.682 {8*a^2+104*a where a = 3.14159 [Pi]}]
[     129.133 {8*a+104 where a = 3.14159 [Pi]},  67.4183 {a^2+18*a+1 where a = 3.14159 [Pi]}]

ca_poly of length 3
[4393.77 {a^4+20*a^3+150*a^2+500*a+625 where a = 3.14159 [Pi]}, -909.633 {-2*a^2-84*a-626 where a = 3.14159 [Pi]}, 1]

ca_mat of size 2 x 2
[0, 0]
[0, 0]

Calcium correctly recognizes singular matrices, even matrices that are nontrivially singular.

In [68]:
A = ca_mat([[pi, pi**2], [pi**3, pi**4]])
A

ca_mat of size 2 x 2
[  3.14159 {a where a = 3.14159 [Pi]}, 9.86960 {a^2 where a = 3.14159 [Pi]}]
[31.0063 {a^3 where a = 3.14159 [Pi]}, 97.4091 {a^4 where a = 3.14159 [Pi]}]

In [69]:
try:
    A.solve(ca_mat([[1], [2]]))
except ZeroDivisionError:
    print("matrix is singular!")

matrix is singular!


In [70]:
A.rref()

ca_mat of size 2 x 2
[1, 3.14159 {a where a = 3.14159 [Pi]}]
[0,                                  0]

In [71]:
A.rank()

1

Exact matrix operations easily result in large expressions:

In [72]:
A = ca_mat([[sqrt(i+j+1) for i in range(6)] for j in range(6)])
A

ca_mat of size 6 x 6
[                                      1, 1.41421 {a where a = 1.41421 [a^2-2=0]},   1.73205 {a where a = 1.73205 [a^2-3=0]},                                         2,   2.23607 {a where a = 2.23607 [a^2-5=0]},   2.44949 {a where a = 2.44949 [a^2-6=0]}]
[1.41421 {a where a = 1.41421 [a^2-2=0]}, 1.73205 {a where a = 1.73205 [a^2-3=0]},                                         2,   2.23607 {a where a = 2.23607 [a^2-5=0]},   2.44949 {a where a = 2.44949 [a^2-6=0]},   2.64575 {a where a = 2.64575 [a^2-7=0]}]
[1.73205 {a where a = 1.73205 [a^2-3=0]},                                       2,   2.23607 {a where a = 2.23607 [a^2-5=0]},   2.44949 {a where a = 2.44949 [a^2-6=0]},   2.64575 {a where a = 2.64575 [a^2-7=0]}, 2.82843 {2*a where a = 1.41421 [a^2-2=0]}]
[                                      2, 2.23607 {a where a = 2.23607 [a^2-5=0]},   2.44949 {a where a = 2.44949 [a^2-6=0]},   2.64575 {a where a = 2.64575 [a^2-7=0]}, 2.82843 {2*a where a = 1.41421 [a^2-2=0]},   

In [ ]:
A.det()

-8.09559e-17 {-4*a*c*e*f-20*a*c*e*g-24*a*c*e-4*a*c*f*g+8*a*c*f+136*a*c-28*a*e*f*g-116*a*e*f-88*a*e*g+64*a*e+112*a*f*g+164*a*f-60*a*g+244*a+204*c*e*f*g-96*c*e*f+8*c*e*g+144*c*e-152*c*f*g-240*c*f+500*c*g+548*c-216*e*f*g+116*e*f+628*e*g+764*e-500*f*g+24*f-1440*g-3868 where a = 3.31662 [a^2-11=0], b = 3.16228 [b^2-10=0], c = 2.64575 [c^2-7=0], d = 2.44949 [d^2-6=0], e = 2.23607 [e^2-5=0], f = 1.73205 [f^2-3=0], g = 1.41421 [g^2-2=0]}

### Eigenvalues and matrix functions

Calcium can calculate exact eigenvalues of matrices, with correct multiplicities. This is accomplished by factoring the characteristic polynomial. Currently, computing polynomial roots will only work for polynomials with very simple structure or with rational entries, so do not expect too much!

In [ ]:
A = ca_mat([[1,pi],[-pi,1]])
display(A)

for lamda, mult in A.eigenvalues():
    print("Multiplicity", mult)
    display(lamda)

ca_mat of size 2 x 2
[                                   1, 3.14159 {a where a = 3.14159 [Pi]}]
[-3.14159 {-a where a = 3.14159 [Pi]},                                  1]

We demonstrate computing the Jordan decomposition of a matrix with nontrivial Jordan form:

In [ ]:
A = ca_mat([[20,77,59,40], [0,-2,-3,-2], [-10,-35,-23,-15], [2,7,3,1]])
J, P = A.jordan_form(transform=True)
display(J)
display(P)
display(P * J * P.inv())

We construct a simple matrix and compute its matrix logarithm, which is uses Jordan decomposition internally:

In [ ]:
A = ca_mat([[-1, -2], [1, 1]])
display(A)
display(A.log())

We evaluate the exponential of the logarithm, recovering the original matrix. This will only work in very simple cases.

In [ ]:
A.log().exp()

Another nice example:

In [ ]:
B = ca_mat([[0,0,1],[0,1,0],[1,0,0]])
display(B.log())
display(B.log().exp())

The following matrix is nilpotent; its exponential is a polynomial expression of the matrix:

In [ ]:
A = ca_mat([[10,32,3,-13], [-8,-30,-4,11], [25,90,11,-34], [-6,-28,-5,9]])
display(A.exp())
display(A**0 + A**1 + A**2/2 + A**3/6)

We construct the 5x5 Hilbert matrix and check that its eigenvalues satisfy the expected determinant and trace relations:

In [ ]:
H = ca_mat([[ca(1)/(i+j+1) for i in range(5)] for j in range(5)])
H

In [ ]:
eig = H.eigenvalues()
for c, mult in eig:
    display(c)

In [ ]:
display(sum(c * mult for (c, mult) in eig)); display(H.trace())
display(prod(c ** mult for (c, mult) in eig)); display(H.det())

## Polynomials

The `ca_poly` type represents univariate polynomials with `ca` coefficients. We can construct polynomials and do arithmetic:

In [ ]:
x = ca_poly([0,1])
f = 1 + (2 + ca(2).sqrt() * ca.pi()) * x + 3*x**2
f

In [ ]:
f ** 5

In [ ]:
ca_poly([1,1,1,1,1,1,1,1,1]).integral()

Polynomial division, GCD and other operations work as expected:

In [ ]:
display(f ** 5 // f**4)

In [ ]:
display(f**5 % f**4)

In [ ]:
(f * (x**2-1)).gcd(f**2 * (x-1))

In [ ]:
(x**2 + ca.pi()**2).gcd(x + ca.i() * ca.pi())

In [ ]:
ca_poly([9,6,7,-28,12]).squarefree_part()

In [ ]:
# Squarefree factorization
for (fac, mult) in ca_poly([9,6,7,-28,12]).factor_squarefree()[1]:
    print(f"Multiplicity {mult}:")
    display(fac)

Finding the roots of a high-degree polynomial:

In [ ]:
f = 4*x**7 + 4*x**6 - 11*x**5 - 16*x**4 + x**3 + 15*x**2 + 10*x + 2

for root, mult in f.roots():
    print(f"Multiplicity {mult}:")
    display(root)

In [ ]:
f(sqrt(2))